In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from __future__ import (print_function, division, absolute_import)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Introduction to Tensorflow 

## Preface
Data engineering makes up for most of the hard work around machine learning. Although we're not attempting to do the data scientists' jobs here, we still need to understand what they're doing to be able to support them best. That's why we'll touch machine learning concepts here. Deliberately, I'll be using *gradient descent* without explaining it. You'll hear about that in later lectures on the subject. What matters here is  that it teaches us about the advanced capabilities available in the chosen technology stack and it provides us a use case that's interesting enough to experience the fun in data engineering.

In part II, I'll be getting into some *feature engineering*, which lives in the border area between machine learning and data engineering.

# Part I: A toy problem
![Baking Powder Machine](../images/baking-powder-packing-machine-500x500.jpg)

Let's assume we are responsible for a machine that produces a particular powder ingredient for baking. And we found it little surprising to hear that the quality of the output depends on the humidity within the last inches of the output pipe. Indeed, at certain humidity levels the probability of the product clumping together or sticking to the pipe increased. It's our job to find out what to do. 

The machine we're talking about has two technical and somewhat mysterious parameters $\beta_1$ and $\beta_2$ that can be measured and tuned. We suspect that the humidity is somehow influenced by these parameters and you are supposed to prove it now. 

A reasonable first hypothesis could be written in mathematical terms as:

$$
h= A_1 \cdot \beta_1 + A_2 \cdot \beta_2 + C
$$

where $h$ is the measured humidity, and $A_1, A_2$ and $C$ are three model parameters that we need to compute now. Any non-zero value for $A_1, A_2$ will confirm our suspicion.

Here's how we take some measurements. Note that we also recorded the weekday and the hour of the day when the measurement took place. At this point in time we consider them irrelevant. In the second part of this exercise though, we'll see that they can actually be meaningfully included in the analysis.

### Data first

```measure(N) ``` creates a [pandas](https://pandas.pydata.org/) data frame with $N$ measurement records in it.

In [ ]:
from measurements import measure

In [ ]:
data = measure(5)
data.head()

In [ ]:
data = measure(10000)
data.describe()

### First Evidence for correlation
Scatter plots are easily created with pandas. We can see that $\beta_1$ has a strong correlation with the humidity and it also exhibits a particularly unusual distribution, as the field has a sharper lower edge. We'll be investigating this in part II of this exercise. With $\beta_2$, there might indeed be a very weak negative correlation, but we can't be certain.

In [ ]:
data = measure(200)
data.plot.scatter(x='beta1', y='humidity')
data.plot.scatter(x='beta2', y='humidity');

### Working with tensors

In Tensorflow, everything is a tensor. A number is a rank-0 tensor - sometimes code-named *scalar*, A vector is a rank-1 tensor, a matrix is a rank-2 tensor and all higher-ranked tensors are just rank-n tensors.

Now, we'll make the hypothesis that the correlations can be described by an *affine* function of those technical parameters $\beta_1$ and $\beta_2$. We simply encode this hypothesis using matrix multiplication:

$$
h(\beta_1, \beta_2) = (A_1, A_2) \cdot
\left( 
\begin {array} {c}
\beta_1 \\
\beta_2
\end{array}
\right) + C
$$

which is equivalent to saying:

$$
h(\beta_1, \beta_2) = A_1 \beta_1 +  A_2 \beta_2 + C
$$

Now all that's left is trying to find $A_1, A_2$ and $C$ such that the above function $h$ best reproduces the actually performed measurements for the humidity.

In [ ]:
import tensorflow as tf

```tf.Variable```s are those tensors that are meant to be tuned during an optimization process. Since we want to tweak and tune $A_1, A_2$ and $C$, we create variables for those.

In [ ]:
# A 1x2 tensor (matrix) for the coefficients A_i
# We're starting with arbitrary values
A = tf.Variable([[1., 2.]], name="coefficients_A", dtype=tf.float32) 
A

In [ ]:
# A scalar tensor for the *bias* C
C = tf.Variable(3., name="bias_C", dtype=tf.float32)
C

You can actually look into those tensors and inspect their values with the help of a session (just ignore the initializer for the time being):

In [ ]:
with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    print(C.eval())
    print(A.eval())

We use ```tf.placeholder```s for the input data and the *labels*. In machine learning, *labels* represent the *true* values. When *labels* are provided, we say that we're doing *supervised learning*. Here, the labels are the true (measured) humidity. And that's what we want the hypothesis to mimic (reproduce). The ```None``` dimension is left open intentionally as it is this dimension that accounts for the number of input records that will be processed at once.

In [ ]:
beta = tf.placeholder(shape=(2,None), name="beta", dtype=tf.float32) 
lbls = tf.placeholder(shape=(1,None), name="true_humidity", dtype=tf.float32)  
beta

### The hypothesis function as a computational graph

The linear humidity hypothesis ```h``` represents a computational graph that we can actually navigate to see its branches and leaves like demonstrated below. 

You'll read the plus operator in the following expression
```
h = tf.matmul(A, beta) + C
```
as: take the tensor to the left, attach it to the left of a "+" node, then take the tensor to the right (C) and attach it to the right side of that node. Return the "+" node and with it the attached computational graph.

```
h=
         +
      /    \ 
 matmul      C
 /    \
A     beta
```

In [ ]:
h = tf.matmul(A, beta) + C
# h represents the + operation: the root of the tree
print(h)
print()

# The matmul operation is the first input of the + operator
print(h.op.inputs[0])
print()

# The coefficients A to the left of the matmul operation:
print(h.op.inputs[0].op.inputs[0])
print()

# The input beta to the left of the matmul operation:
print(h.op.inputs[0].op.inputs[1])

---


#### Evaluating the graph

We use a Tensorflow session object to evaluate the computational graph at the particular values given by ```beta_input```

In [ ]:
beta_input = [list(data['beta1']), list(data['beta2'])]

In [ ]:
beta_input[0][:5]

#### Executing the hypothesis
It's always the same: Initialize all variables and then run the session with the desired tensor node (or a list of tensors). The graph will then resolve the dependencies transitively, compute those and work back to the root where the final result is then computed and passed as a regular number or numpy array. Resolving the placeholders required us to provide some input, which we did using the so-called ```feed_dict```. 

In [ ]:
init = tf.global_variables_initializer()
with tf.Session() as session:
    session.run(init)
    model_humidity = session.run(h, feed_dict={beta: beta_input })

In [ ]:
true_humidity = list(data['humidity'])
true_humidity[:5]

In [ ]:
zip(true_humidity, list(model_humidity.squeeze()))[:5]

The results differ vastly from the true humidity - unsurprisingly so - we haven't tuned the parameters $A$ and $C$ yet.  

#### Delegating execution specifics to the session
Note that the complexity seen above makes sense because once the graph is constructed it's up to the Tensorflow runtime to find out how to compute the results in the most efficient way, taking also the available advanced system resources, such as GPUs or TPUs into account. You as a developer need not bother, just pass your graph to the session.

#### All input in one go
Note also that all the records will processed in a single go. There is no loop over the input data. Indeed all the input data forms a single $N \times 2$ matrix with $N$ being the number of samples. That makes computation truly efficient, as GPUs and TPUs can perform thousands of mathematical operations in parallel, and there's no forth and back between the Python script interpreter and the underlying computational infrastructure.

#### Error = Distance between model and reality

We want the model humidity to be as close as possible to the *true* humidity. To achieve that we need to measure how close the model has come to the true humidity. The mean squared error is a good candidate for that purpose, and we'll use the placeholder ```lbls``` ('labels' is commonly used in machine learning) that we introduced previously to represent the true humidity.

In [ ]:
msq_error = tf.losses.mean_squared_error(h,lbls)

# We need to fit the shape to the hypothesis input placeholder
true_humidity = [true_humidity] 

In [ ]:
with tf.Session() as session:
    session.run(init)
    error = session.run(msq_error, 
                        feed_dict={
                            beta: beta_input, 
                            lbls: true_humidity })
print(error)

#### Tuning the parameters

*Note: We'll use some advanced concepts that we can't cover to the full extent here. Please refer to any of the quazillions of amazing introductory sources available on the internet to get familiar with the concept of gradient descent. [This video by 3Blue1Brown talks about gradient descent in the concept of neural networks, but still it's super-easy to grasp because of its amazing visualizations](https://www.youtube.com/watch?v=IHZwWFHWa-w&t=1s)*.

The *gradient* is a measure of how much the value of a function changes, when its input changes infinitesimaly. If e.g. the gradient of the distance with respect to $A_1$ is positive then that means making $A_1$ a bit smaller would also make the distance smaller. And that's exactly what we're trying to achieve. So what we'll do is iteratively substract a fraction (defined by some small learning rate epsilon $\varepsilon$) of the gradient from the values of $A$ and $B$. And we do this in vector form.

In [ ]:
epsilon = 1e-2

In [ ]:
grad_A = tf.gradients(msq_error, A)
grad_C = tf.gradients(msq_error, C)

Note how easy it is to calculate gradients here. It's all built into the framework. That's super-easy here, because the computational graph allows the computation of any derivative from basic principles according to the rules that we all (should've) learned in our calculus classes.

In [ ]:
Tune_A = tf.assign_add( A, tf.multiply(grad_A[0], -epsilon))
Tune_C = tf.assign_add( C, tf.multiply(grad_C[0], -epsilon))

In mathematical terms, what we do is:
$$
A \leftarrow A - \varepsilon \cdot \frac{\partial}{\partial A} \text{MSE}
$$

$$
C \leftarrow C - \varepsilon \cdot \frac{\partial}{\partial C} \text{MSE}
$$

with MSE being the mean squared error ```msq_error```. Computationally, evaluating ```Tune_A``` will have the *side effect* of changing the value of variable ```A``` and ```Tune_C``` will do that with ```C```.

We create some independent test data

In [ ]:
test_data = measure(20)
test_true = [list(test_data['humidity'])]
test_beta = [list(test_data['beta1']), list(test_data['beta2'])]

...and the data that we use for the optimization process:

In [ ]:
data = measure(10000)
true_humidity = [list(data['humidity'])]
beta_input = [list(data['beta1']), list(data['beta2'])]

Note that the below implementation is deliberately simple and a little inefficient to keep things more readable. The code cell below computes the gradients and adjusts the parameters 200 times and spits out a list of all values for ```msq_error```. 

In [ ]:
errors = []
with tf.Session() as session:
    session.run(init)
    for count in range(200):
        
        # compute the current error/distance
        error = session.run(msq_error, feed_dict={
            beta: beta_input, 
            lbls: true_humidity 
        })
        errors.append(error)
        
        # tune and tweak the parameters a little
        session.run([Tune_A, Tune_C], feed_dict={
            beta: beta_input, 
            lbls: true_humidity 
        })

    # Storing test results and parameters
    test_results, a, c = session.run([h, A, C], feed_dict={
        beta: test_beta})
        
print(errors)

The steadily decreasing list of errors above is a very welcome sign. It means: Our model's output values are indeed getting closer and closer to the *true* humidity. I dare say our model is successfully converging to the *truth*. And we should definitely see that when we explicitly compare true humidity with the one that our model *predicts*.

In [ ]:
zip(list(test_results[0]), test_true[0])


The parameters $A_1, A_2$ and $C$ have eventually converged to:

In [ ]:
list(a[0]), c

That concludes the first part of this exercise. You learned to construct and execute computational graphs with Tensorflow and apply one of the most fundamental techniques, namely gradient descent to solve an analytical problem. 

---

# Part II: Feature Engineering

So, are we happy? To a large degree we could indeed be. We have found convincing evidence that those
mysterious parameters indeed have some influence on the humidity, so tuning them appropriately may help overcome the afforementioned problems. Yet, we began to suspect that something else is wrong. To provide some more evidence, let's have a look at the distribution of the errors over a larger data set.


In the following context, we'll be working with the parameters $A_1, A_2, B$ that we've found previously and thus make them constants. For $10000$ measurements we record how far we have predicted from the measured humidity.

In [ ]:
data = measure(10000)
true_humidity = [list(data['humidity'])]
beta_input = [list(data['beta1']), list(data['beta2'])]

A = tf.constant([2, -.5], shape=(1,2), dtype=tf.float32)
C = tf.constant(18.5, shape=(), dtype=tf.float32)
h = tf.matmul(A, beta) + C
error = (h-lbls)
with tf.Session() as session:
    errors = session.run(
        error, feed_dict={beta: beta_input, lbls: true_humidity})

In [ ]:
pd.DataFrame(errors[0]).hist(bins=30);

What a surprise! Error distributions of that kind indicate that the linear model is overlooking something. It appears ok for the majority of measurements (the big peak around 0), but a small subset of measurements is apparently influenced by still unknown forces. 

Now, how to approach that problem? At the moment, we're only considering $\beta_1$ and $\beta_2$ as influence factors. Maybe there are other factors. Maybe on certain days or at certain times of the day the humidity is influenced by other events?


In [ ]:
import seaborn as sns
sns.set()
data = measure(2000)

```seaborn``` introduces a convenience layer on top of matplotlib using pandas under the hood. It's extremely helpful for quick explorations.

In [ ]:
sns.catplot(x='weekday', y='humidity', kind='bar', data=data);

Look at the average humidity for each day of the week. Still hard to say. Thursdays and Sundays appear particularly low. But do we have sufficient statistics to conclude? Let's have a look at the hours of the day:

In [ ]:
sns.catplot(x='hour', y='humidity', kind='bar', data=data);

The error bars are not big enough to dismiss the differences as statistical noise. So we're getting closer. But still it's not easy to tell what's really going on. 

*(Hint: Don't be shy to look at the statistics of 50'000 data points)*

Well, here we are. It's obvious that we need to take weekdays and hours of day into account. But that's not so trivial. While $\beta_1$ and $\beta_2$ were suspected to influence the humidity somewhat linearly, that is certainly not true for hour and weekday. Data engineering has the answer: *one-hot* encoding. That's almost always a successful strategy, if it is somehow a *yes-or-no* or a *one-in-N* aspect that governs the problem.

---
#### One-Hot encoding weekdays

We use a neat combination of Python tricks to one-hot encode a number: The $n$-th column (or row) vector of the identity matrix (```np.eye``` in the numpy world), is the one-hot encoding for $n$:

In [ ]:
ID = np.eye(7)
ID

In [ ]:
for n in [0, 3, 6]:
    print("%s -> %s" % (n, ID[n]))

You see the $1$s at the position given by the index. So, Monday would be represented by the first vector, Thu by the second and Sun by the third.

Here comes the even cooler trick: We can use an array of indices rather than single indices one by one:

In [ ]:
indices = [0,3,6]
ID[indices]

See: All three days one-hot-encoded in a single go

---
#### Preparing data for the next round
We'll need a full dataset with one-hot encoded weekdays.

In [ ]:
data = measure(10000)
data[:5]

In [ ]:
# This does one hot encoding and gets the shape right.
def one_hot(df, size):
    ords = list(df)
    return list(np.transpose(np.eye(size)[ords]))

Putting the one-hot encoded values in a data frame. That's not necessary, but seeing it this way makes it much clearer what one-hot encoding actually means

In [ ]:
days_one_hot = one_hot(data['weekday'], 7)
weekdays=['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
for i in range(7):
    data[weekdays[i]]=days_one_hot[i]
data[:5]

Now we have $2+7=9$ different input *signals*, and we'll use a $1 \times 9$ parameter matrix to deal with them.

In [ ]:
def create_input(data):
    days_one_hot = one_hot(data['weekday'], 7)
    weekdays=['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
    for i in range(7):
        data[weekdays[i]]=days_one_hot[i]
    all_input = [
        list(data['beta1']), 
        list(data['beta2']),
        list(data['Mon']),
        list(data['Tue']),
        list(data['Wed']),
        list(data['Thu']),
        list(data['Fri']),
        list(data['Sat']),
        list(data['Sun'])
    ]
    return all_input

true_humidity = [list(data['humidity'])]
all_input = create_input(data)

test_data = measure(10000)
test_input = create_input(test_data)
test_humidity = [list(test_data['humidity'])]

In [ ]:
test_data[:5]

---
# Exercise
Write the code for the new hypothesis function like below:


$$
h(...) = (A_1, A_2, A_m, A_t, A_w, A_{th}, A_f, A_s, A_{su}) \cdot
\left( 
\begin {array} {c}
\beta_1 \\
\beta_2 \\
\text{Mon} \\
\text{Tue} \\
\text{Wed} \\
\text{Thu} \\
\text{Fri} \\
\text{Sat} \\
\text{Sun}
\end{array}
\right) + C
$$

In [ ]:
#    Your turn: Replace the None placeholder with meaningful code.
#
# A 1x9 tensor (matrix) for the coefficients A_i
# Start with arbitrary (but small) values
A = None 
A

In [ ]:
#    Nothing new here
#
# A scalar tensor for the *bias* C
C = None
C

In [ ]:
#
#  Placeholders shape=(9, None) for input data, (1, None) for labels
#
input_placeholder = None 
labels_placeholder = None
input_placeholder

In [ ]:
#    Thanks to the vector notation, the hypothesis function doesn't 
#    change
#
h = None
h

In [ ]:
#
#   Mean squared error: Nothing changes here
#
msq_error = None
msq_error

In [ ]:
#
#   gradients and updating: Nothing changes here
#
grad_A = None
grad_C = None
Tune_A = None
Tune_C = None
Tune_A

In [ ]:
errors = []
with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    for count in range(200):
        
        # compute the current error/distance
        error = session.run(msq_error, feed_dict={
            input_placeholder: all_input, 
            labels_placeholder: true_humidity 
        })
        errors.append(error)
        
        # tune and tweak the parameters a little
        session.run([Tune_A, Tune_C], feed_dict={
            input_placeholder: all_input, 
            labels_placeholder: true_humidity 
        })

    # Storing test results and parameters
    test_results, a, c = session.run([h, A, C], feed_dict={
        input_placeholder: test_input})
        
print(errors)

In [ ]:
a, c

Look at the parameters $A_{Thu}$ (6th) and $A_{Sun}$ (9th). These are lower than the other weekdays. The model found out that the humidity is slightly lower on these days. That's a little bit of an achievement. Although, when we look at the error distribution like we did before:

In [ ]:
A = tf.constant(a, shape=(1,9), dtype=tf.float32)
C = tf.constant(c, shape=(), dtype=tf.float32)
h = tf.matmul(A, input_placeholder) + C
error = (h-labels_placeholder)
with tf.Session() as session:
    errors = session.run(
        error, feed_dict={
            input_placeholder: test_input, 
            labels_placeholder: test_humidity})

In [ ]:
pd.DataFrame(errors[0]).hist(bins=30);

We can see that the weekday together with the $\beta$ parameters still don't explain the humidity observations sufficiently. As a matter of fact, if you also hot-encode the hours of the day and perform a feature crossing to produce a 168-dimensional hour-of-the-week feature, then you'll see the bump disappear and you'll find that the model *learns to understand* that there's additional humidity at certain hours of the week. Give it a try!

In [ ]:
#    Your turn
#
# A 1x9 tensor (matrix) for the coefficients A_i
# Start with arbitrary (but small) values
A = tf.Variable([[1,1,0,0,0,0,0,0,0]], 
                name="coefficients_A", dtype=tf.float32) 
A

In [ ]:
#    Nothing new here
#
# A scalar tensor for the *bias* C
C = tf.Variable(3., name="bias_C", dtype=tf.float32)
C

In [ ]:
#
#  Placeholders shape=(9, None) for input data, (1, None) for labels
#
input_placeholder = tf.placeholder(
    shape=(9,None), name="all_input", dtype=tf.float32) 
labels_placeholder = tf.placeholder(
    shape=(1,None), name="true_humidity", dtype=tf.float32)  
input_placeholder

In [ ]:
#    Thanks to the vector notation, the hypothesis function doesn't 
#    change
#
h = tf.matmul(A, input_placeholder) + C
h

In [ ]:
#
#   Mean squared error: Nothing changes here
#
msq_error = tf.losses.mean_squared_error(h,labels_placeholder)
msq_error

In [ ]:
#
#   gradients and updating: Nothing changes here
#
grad_A = tf.gradients(msq_error, A)
grad_C = tf.gradients(msq_error, C)
Tune_A = tf.assign_add( A, tf.multiply(grad_A[0], -epsilon))
Tune_C = tf.assign_add( C, tf.multiply(grad_C[0], -epsilon))
Tune_A